In [12]:
import json
import pandas as pd
import urllib.request

In [13]:
# print all the tables
titles = {
    'M182931': 'Average Monthly Nominal Earnings Per Employee, Quarterly',
    'M920131': 'Median Gross Monthly Income From Work (Including Employer CPF) Of Full-Time Employed Residents By Occupations And Sex,(June), Annual',
    'M182981': 'Gross Monthly Income From Work (Including Employer CPF) Of Full-Time Employed Residents, Annual',
    'M183131': 'Changes In Employment By Sector, (Compared To The Previous Period), Quarterly',
    'M182342': 'Unemployment Rate (End Of Period), Quarterly, Seasonally Adjusted',
    'M015701': 'Changes In Value Added Per Worker In Chained (2015) Dollars, By Industry (SSIC 2020), Quarterly',
    'M183742': 'Unit Labour Cost Index (2015 = 100), Quarterly, Seasonally Adjusted',
    'M182331': 'Labour Force Aged 15 Years And Over, End June, Annual',
    'M920151': 'Number Of Part-Time Employed Residents By Sex, (June), Annual',
    'M920141': 'Number Of Full-Time Employed Residents By Sex, (June), Annual',
    'M920161': 'Number Of Employers By Sex, (June), Annual',
    'M181641': 'Labour Market Statistics (End Of Period), Quarterly'
    }
urlData = {
    'M182931': 'https://tablebuilder.singstat.gov.sg/api/table/tabledata/M182931?isTestApi=true',
    'M920131': 'https://tablebuilder.singstat.gov.sg/api/table/tabledata/M920131?isTestApi=true',
    'M182981': 'https://tablebuilder.singstat.gov.sg/api/table/tabledata/M182981?isTestApi=true',
    'M183131': 'https://tablebuilder.singstat.gov.sg/api/table/tabledata/M183131?isTestApi=true',
    'M182342': 'https://tablebuilder.singstat.gov.sg/api/table/tabledata/M182342?isTestApi=true',
    'M015701': 'https://tablebuilder.singstat.gov.sg/api/table/tabledata/M015701?isTestApi=true',
    'M183742': 'https://tablebuilder.singstat.gov.sg/api/table/tabledata/M183742?isTestApi=true',
    'M182331': 'https://tablebuilder.singstat.gov.sg/api/table/tabledata/M182331?isTestApi=true',
    'M920151': 'https://tablebuilder.singstat.gov.sg/api/table/tabledata/M920151?isTestApi=true',
    'M920141': 'https://tablebuilder.singstat.gov.sg/api/table/tabledata/M920141?isTestApi=true',
    'M920161': 'https://tablebuilder.singstat.gov.sg/api/table/tabledata/M920161?isTestApi=true',
    'M181641': 'https://tablebuilder.singstat.gov.sg/api/table/tabledata/M181641?isTestApi=true'
    }
tables = {}

In [14]:
for item in urlData.items():
    webURL = urllib.request.urlopen(item[1])
    data = webURL.read()

    df = json.loads(data)
    df_converted = pd.json_normalize(df['Data']['row'][0]['columns'])
    tables[item[0]] = df_converted

    # title
    print('Title: ' + df['Data']['title'] + '\n')
    # df
    print(df_converted)
    # footnote
    print('\nFootnote:\n' + df['Data']['footnote'])
    # data last updated 
    print('\nData Last Updated: ' + df['Data']['dataLastUpdated'] + '\n\n\n\n\n')

Title: Average Monthly Nominal Earnings Per Employee, Quarterly

        key value
0   2001 2Q  2919
1   2001 3Q  2963
2   2001 4Q  3463
3   2002 1Q  3206
4   2002 2Q  2931
..      ...   ...
80  2021 2Q  5474
81  2021 3Q  5412
82  2021 4Q  6282
83  2022 1Q  6641
84  2022 2Q  5847

[85 rows x 2 columns]

Footnote:
1) Average (mean) monthly earnings refer to all remuneration received before deduction of the employee Central Provident Fund (CPF) contributions and personal income tax.  It comprises basic wages, overtime pay, commissions, allowances and bonuses but exclude employer CPF contributions.  2) Data cover full-time and part-time employees who have CPF contributions.  They exclude identifiable self employed persons who have made CPF contribution.  3) For details of data limitation, please refer to note 3 of Table B.9 in the Singapore Yearbook of Manpower Statistics, 2016 (http://stats.mom.gov.sg/iMAS_Tables/YearBook/YearBook_2016/mrsd_2016YearBook_WtableB_9.xlsx)

Data Last Updated

In [15]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

groups = [
    ('M182931', 'M920131', 'M182981'), 
    ('M183131', 'M182342'),
    ('M015701', 'M183742'),
    ('M182331', 'M920151', 'M920141', 'M920161'),
          ]

categories = (
    'Income',
    'Employment',
    'Efficiency',
    'Demographics'
)

figs = []

for table_groups in groups:
    fig = make_subplots(rows = len(table_groups), cols = 1, vertical_spacing = .1)
    for row, table in enumerate(table_groups, start=1):
        fig.add_scatter(x=tables[table]['key'],
                        y=tables[table]['value'],
                        mode='lines+markers',
                        name=titles[table],
                        row=row,
                        col=1)
        
    fig.update_layout(height=900, 
                width=1200,
                title = {
                    'text': categories[groups.index(table_groups)],
                    'x': 0.5,
                    'y': 0.95,
                    'font': {
                        'color': 'black'
                    }
                    },
                legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.0,
                    xanchor="right",
                    x=0.8
                    ),
                margin=dict(l=20, r=20, t=150, b=20),
                autotypenumbers='convert types'
                )
    fig.update_layout(
    autosize=True,
    )

        
    figs.append(fig)

for fig in figs:
    fig.show()

In [20]:
from jupyter_dash import JupyterDash
import dash_html_components as html

title = html.H2(
                'Singapore Manpower Statistics',
                style = {
                    'backgroundColor': 'tan',
                    'fontFamily': 'verdana',
                    'textAlign': 'center',
                    'height': '900',
                    'width': '1200'
                    }
                )
# link = html.A('See Dashboard', href ='https://manpowerstats.yongkheehou.com')
# layout = html.Div([title, link])

In [17]:
for category in categories:
    print(category)

Income
Employment
Efficiency
Demographics


In [21]:
import dash_core_components as dcc

def create_tab(content, label, value):
    return dcc.Tab(
        content,
        label=label,
        value=value,
        id=f'{value}-tab',
        className='single-tab',
        selected_className='single-tab--selected'
    )

In [22]:
income_graph = dcc.Graph(figure=figs[0], id='income_graph')
employment_graph = dcc.Graph(figure=figs[1], id='employment_graph')
efficiency_graph = dcc.Graph(figure=figs[2], id='efficiency_graph')
demographics_graph = dcc.Graph(figure=figs[0], id='demographics_graph')

income_tab = create_tab(income_graph, 'Income', 'income')
employment_tab = create_tab(employment_graph, 'Employment', 'employment')
efficiency_tab = create_tab(efficiency_graph, 'Efficiency', 'efficiency')
demographics_tab = create_tab(demographics_graph, 'Demographics', 'demographics')

graph_tabs = dcc.Tabs(
    [income_tab, employment_tab, efficiency_tab, demographics_tab],
    className='tabs-container',
    id='graph-tabs',
    value='cumulative',
    style={'gridArea': 'graphs', 'margin': '0px'}
)

layout = html.Div([title, graph_tabs])

app = JupyterDash(__name__)
app.layout = layout
app.run_server(mode='inline', height = 100)